In [2]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import datetime

In [3]:
GPS_DATA_FOLDER = Path("../BDC_30_routes_April_2025/raw_GPS")
CLEAN_GPS_DATA_FOLDER = Path("../BDC_30_routes_April_2025/clean_GPS")
PROCESSED_GPS_DATA_FOLDER = Path("../BDC_30_routes_April_2025/processed_raw_GPS")
PROCESSED_GPS_DATA_FOLDER.mkdir(parents=True, exist_ok=True)

# Định nghĩa hàm xử lý dữ liệu cho từng file

In [4]:
vehicle_col = "anonymized_vehicle"
driver_col = "anonymized_driver"
lat_col = "lat"
lng_col = "lng"
time_col = "datetime"

In [5]:
def process_data(df: pd.DataFrame, day: int) -> pd.DataFrame:
    """Apply all cleaning and transformation steps for a single raw GPS dataframe."""
    processed = df.copy()

    # Chuyển đổi kiểu dữ liệu
    processed[time_col] = pd.to_datetime(processed.loc[:, time_col])
    processed[lat_col] = pd.to_numeric(processed.loc[:, lat_col], errors="coerce")
    processed[lng_col] = pd.to_numeric(processed.loc[:, lng_col], errors="coerce")

    print("Số lượng bản ghi ban đầu:", len(df))
    processed = processed.drop_duplicates()
    print("Số lượng bản ghi sau khi loại bỏ trùng lặp:", len(processed))

    # Gioi hạn thoi gian hoạt động của xe trong ngày từ 5h sáng đến 23h tối
    selected_time_window = (datetime.datetime(2025, 4, day, 5, 0, 0), datetime.datetime(2025, 4, day, 23, 0, 0))
    processed = processed.loc[(processed[time_col] >= selected_time_window[0]) & (processed[time_col] <= selected_time_window[1])]
    print(f"Số bản ghi sau khi lọc theo khoảng thời gian {selected_time_window[0]} đến {selected_time_window[1]}: {len(processed)}")

    # Sắp xếp lại dữ liệu theo từng xe và thời gian ghi nhận
    processed = processed.sort_values(by=[vehicle_col, driver_col, time_col], ascending=[True, True, True])

    # Giảm số lượng lấy mẫu, mỗi phút chỉ lấy một mẫu (giữ record đầu tiên trong phút)
    processed["time_minute"] = processed[time_col].dt.floor("1min") # Tạo cột mới theo thời gian chỉ lấy đến phút
    processed = processed.set_index("time_minute")
    resampled_frames = []
    agg_dict = {
        lat_col: "first",
        lng_col: "first",
        "speed": "first",
        "door_up": lambda x: x.astype(bool).any(),
        "door_down": lambda x: x.astype(bool).any(),
        vehicle_col: "first",
        driver_col: "first",
    }

    for vehicle_id, group in processed.groupby([vehicle_col]):
        group = group.sort_index()
        resampled = group.resample("1min").agg(agg_dict).dropna(subset=[lat_col, lng_col])
        resampled_frames.append(resampled.reset_index())

    processed = pd.concat(resampled_frames, ignore_index=True)
    processed["door_up"] = processed["door_up"].astype(bool)
    processed["door_down"] = processed["door_down"].astype(bool)
    
    print("Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu:", len(processed))
    return processed


# Thực hiện xử lý từng file

In [6]:
csv_files = sorted(GPS_DATA_FOLDER.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {GPS_DATA_FOLDER}")

NUMBER_OF_FILES_TO_PROCESS = 15  # Thay đổi số này để giới hạn số file được xử lý
for idx, csv_path in enumerate(csv_files[:NUMBER_OF_FILES_TO_PROCESS]):
    print(f"Processing {csv_path.name}...")
    df = pd.read_csv(csv_path)
    processed_df = process_data(df, day=idx+1)

    output_path = PROCESSED_GPS_DATA_FOLDER / csv_path.name
    processed_df.to_csv(output_path, index=False)


Processing anonymized_raw_2025-04-01.csv...
Số lượng bản ghi ban đầu: 2354942
Số lượng bản ghi ban đầu: 2354942
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2343951
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-01 05:00:00 đến 2025-04-01 23:00:00: 2056887
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2343951
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-01 05:00:00 đến 2025-04-01 23:00:00: 2056887
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 459944
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 459944
Processing anonymized_raw_2025-04-02.csv...
Processing anonymized_raw_2025-04-02.csv...
Số lượng bản ghi ban đầu: 2262539
Số lượng bản ghi ban đầu: 2262539
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2254199
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-02 05:00:00 đến 2025-04-02 23:00:00: 1981927
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2254199
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-02 05:00:00 đến 2025-04-02 23:00:00: 1981927
Số lượ

Processing anonymized_raw_2025-04-01.csv...
Số lượng bản ghi ban đầu: 2354942
Số lượng bản ghi ban đầu: 2354942
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2343951
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-01 05:00:00 đến 2025-04-01 23:00:00: 2056887
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2343951
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-01 05:00:00 đến 2025-04-01 23:00:00: 2056887
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 459944
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 459944
Processing anonymized_raw_2025-04-02.csv...
Processing anonymized_raw_2025-04-02.csv...
Số lượng bản ghi ban đầu: 2262539
Số lượng bản ghi ban đầu: 2262539
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2254199
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-02 05:00:00 đến 2025-04-02 23:00:00: 1981927
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2254199
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-02 05:00:00 đến 2025-04-02 23:00:00: 1981927
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 450830
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 450830
Processing anonymized_raw_2025-04-03.csv...
Processing anonymized_raw_2025-04-03.csv...
Số lượng bản ghi ban đầu: 2275931
Số lượng bản ghi ban đầu: 2275931
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2263142
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-03 05:00:00 đến 2025-04-03 23:00:00: 1988211
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2263142
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-03 05:00:00 đến 2025-04-03 23:00:00: 1988211
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 448559
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 448559
Processing anonymized_raw_2025-04-04.csv...
Processing anonymized_raw_2025-04-04.csv...
Số lượng bản ghi ban đầu: 2270348
Số lượng bản ghi ban đầu: 2270348
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2262445
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-04 05:00:00 đến 2025-04-04 23:00:00: 1991937
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2262445
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-04 05:00:00 đến 2025-04-04 23:00:00: 1991937
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 449356
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 449356
Processing anonymized_raw_2025-04-05.csv...
Processing anonymized_raw_2025-04-05.csv...
Số lượng bản ghi ban đầu: 2235339
Số lượng bản ghi ban đầu: 2235339
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2225662
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-05 05:00:00 đến 2025-04-05 23:00:00: 1961245
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2225662
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-05 05:00:00 đến 2025-04-05 23:00:00: 1961245
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 447703
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 447703
Processing anonymized_raw_2025-04-06.csv...
Processing anonymized_raw_2025-04-06.csv...
Số lượng bản ghi ban đầu: 2146623
Số lượng bản ghi ban đầu: 2146623
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2133128
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-06 05:00:00 đến 2025-04-06 23:00:00: 1864562
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2133128
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-06 05:00:00 đến 2025-04-06 23:00:00: 1864562
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 434005
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 434005
Processing anonymized_raw_2025-04-07.csv...
Processing anonymized_raw_2025-04-07.csv...
Số lượng bản ghi ban đầu: 2184897
Số lượng bản ghi ban đầu: 2184897
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2172979
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-07 05:00:00 đến 2025-04-07 23:00:00: 1904241
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2172979
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-07 05:00:00 đến 2025-04-07 23:00:00: 1904241
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 442549
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 442549
Processing anonymized_raw_2025-04-08.csv...
Processing anonymized_raw_2025-04-08.csv...
Số lượng bản ghi ban đầu: 2204767
Số lượng bản ghi ban đầu: 2204767
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2184982
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-08 05:00:00 đến 2025-04-08 23:00:00: 1925270
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2184982
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-08 05:00:00 đến 2025-04-08 23:00:00: 1925270
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 443665
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 443665
Processing anonymized_raw_2025-04-09.csv...
Processing anonymized_raw_2025-04-09.csv...
Số lượng bản ghi ban đầu: 2212950
Số lượng bản ghi ban đầu: 2212950
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2167123
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-09 05:00:00 đến 2025-04-09 23:00:00: 1901024
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2167123
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-09 05:00:00 đến 2025-04-09 23:00:00: 1901024
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 438305
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 438305
Processing anonymized_raw_2025-04-10.csv...
Processing anonymized_raw_2025-04-10.csv...
Số lượng bản ghi ban đầu: 2188309
Số lượng bản ghi ban đầu: 2188309
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2171762
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-10 05:00:00 đến 2025-04-10 23:00:00: 1914834
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2171762
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-10 05:00:00 đến 2025-04-10 23:00:00: 1914834
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 442800
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 442800
Processing anonymized_raw_2025-04-11.csv...
Processing anonymized_raw_2025-04-11.csv...
Số lượng bản ghi ban đầu: 2213211
Số lượng bản ghi ban đầu: 2213211
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2204655
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-11 05:00:00 đến 2025-04-11 23:00:00: 1931489
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2204655
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-11 05:00:00 đến 2025-04-11 23:00:00: 1931489
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 444830
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 444830
Processing anonymized_raw_2025-04-12.csv...
Processing anonymized_raw_2025-04-12.csv...
Số lượng bản ghi ban đầu: 2201707
Số lượng bản ghi ban đầu: 2201707
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2193388
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-12 05:00:00 đến 2025-04-12 23:00:00: 1929905
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2193388
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-12 05:00:00 đến 2025-04-12 23:00:00: 1929905
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 441011
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 441011
Processing anonymized_raw_2025-04-13.csv...
Processing anonymized_raw_2025-04-13.csv...
Số lượng bản ghi ban đầu: 2114707
Số lượng bản ghi ban đầu: 2114707
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2105789
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2105789
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-13 05:00:00 đến 2025-04-13 23:00:00: 1847448
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-13 05:00:00 đến 2025-04-13 23:00:00: 1847448
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 425720
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 425720
Processing anonymized_raw_2025-04-14.csv...
Processing anonymized_raw_2025-04-14.csv...
Số lượng bản ghi ban đầu: 3063884
Số lượng bản ghi ban đầu: 3063884
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2757472
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-14 05:00:00 đến 2025-04-14 23:00:00: 2502483
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2757472
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-14 05:00:00 đến 2025-04-14 23:00:00: 2502483
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 441871
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 441871
Processing anonymized_raw_2025-04-15.csv...
Processing anonymized_raw_2025-04-15.csv...
Số lượng bản ghi ban đầu: 2205589
Số lượng bản ghi ban đầu: 2205589
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2192347
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2192347
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-15 05:00:00 đến 2025-04-15 23:00:00: 1931441
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-15 05:00:00 đến 2025-04-15 23:00:00: 1931441
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 441635
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 441635

# Gộp toàn bộ dữ liệu gps vào toàn bộ một file

In [7]:
processed_csv_files = sorted(GPS_DATA_FOLDER.glob("*.csv"))
if not processed_csv_files:
    raise FileNotFoundError(f"No CSV files found in {GPS_DATA_FOLDER}")

dataframes = (pd.read_csv(f) for f in processed_csv_files)
combined_df = pd.concat(dataframes, ignore_index=True)


C:\Users\likgn\AppData\Local\Temp\ipykernel_9592\926853952.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = (pd.read_csv(f) for f in processed_csv_files)
C:\Users\likgn\AppData\Local\Temp\ipykernel_9592\926853952.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = (pd.read_csv(f) for f in processed_csv_files)
C:\Users\likgn\AppData\Local\Temp\ipykernel_9592\926853952.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = (pd.read_csv(f) for f in processed_csv_files)


In [8]:
combined_output_path = PROCESSED_GPS_DATA_FOLDER / "processed_raw_GPS_all_days.csv"
combined_df.to_csv(combined_output_path, index=False)

In [9]:
test_df = pd.read_csv(combined_output_path)
print(f"Số lượng dòng dữ liệu tổng cộng: {len(test_df)}")

C:\Users\likgn\AppData\Local\Temp\ipykernel_9592\1698421680.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(combined_output_path)


KeyboardInterrupt: 

In [ ]:
print(f"Số lượng dòng dữ liệu tổng cộng: {len(combined_df)}")